In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
df = pd.read_csv("/kaggle/input/persona-1/persona_1.csv")

df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [2]:
##### GOREV - 1 #####
# Soru 1: persona.csv dosyasını okutunuz ve veri seti ile ilgili genel bilgileri gösteriniz.

df.shape
df.info()
df.columns
df.index
df.describe().T
df.isnull().values.any()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


False

In [3]:
# Soru 2: Kaç unique SOURCE vardır? Frekansları nedir

df["SOURCE"].unique()
df["SOURCE"].nunique()
df["SOURCE"].value_counts()

SOURCE
android    2974
ios        2026
Name: count, dtype: int64

In [4]:
# Soru 3: Kaç unique PRICE vardır?
df["PRICE"].nunique()

6

In [5]:
# Soru 4: Hangi PRICE'dan kaçar tane satış gerçekleşmiş
df["PRICE"].value_counts()

PRICE
29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: count, dtype: int64

In [6]:
# Soru 5: Hangi ülkeden kaçar tane satış olmuş?
df["COUNTRY"].value_counts()

COUNTRY
usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: count, dtype: int64

In [7]:
# Soru 6: Ülkelere göre satışlardan toplam ne kadar kazanılmış
df.groupby("COUNTRY").agg({"PRICE" : "sum"})

,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


In [8]:
# Soru 7: SOURCE türlerine göre satış sayıları nedir --???
df["SOURCE"].value_counts()


SOURCE
android    2974
ios        2026
Name: count, dtype: int64

In [9]:
# Soru 8: Ülkelere göre PRICE ortalamaları nedir?
df.groupby("COUNTRY").agg({"PRICE" : "mean"})

,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


In [10]:
# Soru 9: SOURCE'lara göre PRICE ortalamaları nedir?
df.groupby("SOURCE").agg({"PRICE" : "mean"})

,PRICE
SOURCE,
android,34.174849
ios,34.069102


In [11]:
# Soru 10: COUNTRY-SOURCE kırılımında PRICE ortalamaları nedir?
df.groupby(["COUNTRY", "SOURCE"]).agg({"PRICE" : "mean"})

PRICE
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

In [12]:
##### GOREV - 2 #####

#  COUNTRY, SOURCE, SEX, AGE kırılımında ortalama kazançlar nedir?
agg_df = df.groupby(["COUNTRY","SOURCE", "SEX", "AGE"]).agg({"PRICE" : "mean"}).sort_values(by="PRICE", ascending=False)

agg_df = agg_df.reset_index()

agg_df["NEW_AGE"] = pd.cut(agg_df["AGE"], [0, 18, 19, 23, 24, 30, 31, 40, 41, 70],
                           labels=['0-18', '18-19', '19-23', '23-24', '24-30',
                                   '30-31', '31-40', '40-41', '41-70'])

agg_df.head()



,COUNTRY,SOURCE,SEX,AGE,PRICE,NEW_AGE
0,bra,android,male,46,59.0,41-70
1,usa,android,male,36,59.0,31-40
2,fra,android,female,24,59.0,23-24
3,usa,ios,male,32,54.0,31-40
4,deu,android,female,36,49.0,31-40


In [13]:
agg_df.head()

agg_df["NEW_AGE"] = [str(i) for i in agg_df["NEW_AGE"]]

agg_df["customers_level_based"] = agg_df["COUNTRY"] +"_"+ agg_df["SOURCE"] +"_"+ agg_df["SEX"] +"_"+ agg_df["NEW_AGE"]
agg_df.head()

last_df = agg_df.groupby("customers_level_based").agg({"PRICE" : "mean"}).sort_values(by="PRICE", ascending=False).reset_index()
last_df.head()


,customers_level_based,PRICE
0,fra_android_female_23-24,59.000000
1,tur_ios_male_23-24,45.000000
2,tur_android_female_31-40,43.000000
3,tur_ios_male_31-40,42.333333
4,fra_ios_male_19-23,42.000000


In [14]:
last_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])
last_df.head()

,customers_level_based,PRICE,SEGMENT
0,fra_android_female_23-24,59.000000,A
1,tur_ios_male_23-24,45.000000,A
2,tur_android_female_31-40,43.000000,A
3,tur_ios_male_31-40,42.333333,A
4,fra_ios_male_19-23,42.000000,A


In [15]:
last_agg_df = last_df.groupby("SEGMENT").agg({"PRICE" : ["mean", "max", "sum"]}).reset_index()
last_agg_df.head()

/tmp/ipykernel_18/4017767617.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  last_agg_df = last_df.groupby("SEGMENT").agg({"PRICE" : ["mean", "max", "sum"]}).reset_index()


SEGMENT      PRICE                   
                mean   max          sum
0       D        NaN   NaN     0.000000
1       C        NaN   NaN     0.000000
2       B  30.292018  34.0  1999.273218
3       A  36.964506  59.0  3141.983050

In [16]:
last_df[last_df["customers_level_based"] == "tur_android_female_31-40"]

,customers_level_based,PRICE,SEGMENT
2,tur_android_female_31-40,43.0,A
